# Testing single node datasets

Purpose: until now dataset represented by combination of tree types - now test for single node type

## Importing

In [1]:
import sys
import os

# Construct the path to the parent directory of the parent directory
parent_of_parent_dir = os.path.abspath(os.path.join(os.path.pardir, os.path.pardir))

# Add the parent directory of the parent directory to the system path
sys.path.insert(0, parent_of_parent_dir)

In [2]:
import optuna
import numpy as np
from sklearn.metrics import r2_score
import torch
from torch.utils.data import random_split, DataLoader

from etnn import TreeNode
from etnn.tools.training_tools import ConfigStore
from etnn.data.ferris_wheel import load_pure_ferris_wheel_dataset_single_node
from etnn.tools.training import train_epoch, eval_epoch
from etnn.nn.layer_framework import LayerManagementFramework
from etnn.tools.training_tools import seeding_all
from etnn.routines.run_config import choice_trainloader, choice_loss, choice_optim

## Control constants

In [3]:
dataset_path = "../../datasets/"
test_perc = 0.3
val_perc = 0.21
stability_count = 5
label = "tree-advanced" # although irrelevant

## Defining parameter search

In [4]:
def objective(trial):
    # init default config
    config = ConfigStore(
       in_dim=15,
       hidden_dim=trial.suggest_int("hidden_dim", 16, 512, step=16),
       out_dim=1,
       k=trial.suggest_int("k", 1, 5),
       dataset=-1 if normalized else 0,
       ds_size=10_000,
       num_gondolas=-1,
       num_part_pg=-1,
       loss_name='mse',
       optimizer_name='adam',
       num_max_epochs=30, # real: 100
       learning_rate=trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True),
       batch_size=1024,
       early_stop_tol=5,
       use_equal_batcher=trial.suggest_categorical("batcher", [True, False]),
       seed=420,
       label_type=label,
       final_label_factor=5/1000
    )
    # config = ConfigStore(
    #     in_dim=15,
    #     hidden_dim=16,
    #     out_dim=1,
    #     k=2,
    #     dataset=-1 if normalized else 0,
    #     ds_size=10_000,
    #     num_gondolas=-1,
    #     num_part_pg=-1,
    #     loss_name='mse',
    #     optimizer_name='adam',
    #     num_max_epochs=30, # real: 100
    #     learning_rate=0.002,
    #     batch_size=1024,
    #     early_stop_tol=5,
    #     use_equal_batcher=False,
    #     seed=420,
    #     label_type=label,
    #     final_label_factor=5/1000
    # )

    # loading dataset
    dataset, df_index = load_pure_ferris_wheel_dataset_single_node(
        node_type=node_type,
        num_elem=size_elem,
        num_to_generate=config.ds_size,
        dataset_path=dataset_path,
        final_label_factor=config.final_label_factor,
        normalize=True if config.dataset == -1 else 0
    )
    # splitting off test dataset
    generator = torch.Generator().manual_seed(config.seed)
    train_ds, val_ds, _ = random_split(
        dataset,
        [1 - test_perc - val_perc, val_perc, test_perc],
        generator=generator
    )

    # loaders
    train_loader = choice_trainloader(config, df_index, train_ds)
    val_loader = DataLoader(val_ds, batch_size=4 * config.batch_size, shuffle=False)

    # define device
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # set seed for reproducability
    seeding_all(config.seed)

    # define model
    model = LayerManagementFramework(
        in_dim=config.in_dim,
        tree=TreeNode(node_type, [TreeNode("E", size_elem)]),
        hidden_dim=config.hidden_dim,
        out_dim=config.out_dim,
        k=config.k
    ).to(device)

    # learning tools
    criterion = choice_loss(config)
    optimizer = choice_optim(config, model)

    # init score list
    score_list = []

    # train for specified number of epochs
    for epoch in range(config.num_max_epochs):
        _, _, _ = train_epoch(
            model,
            train_loader,
            optimizer,
            device,
            criterion
        )

        _, val_true_y, val_pred_y = eval_epoch(
            model,
            val_loader,
            device,
            criterion
        )

        # calc r2 score and append
        score = r2_score(y_true=val_true_y, y_pred=val_pred_y)
        score_list += [score]
        trial.report(score, epoch)

    # calculate objective
    # display(score_list)
    # idea: last x r2 scores (why not last one? for stability purposes)
    obj = np.array(score_list)[-stability_count:]
    return np.mean(obj)

## Parameter search

### S

In [5]:
node_type = "S"
size_elem = 50
n_trials = 25
normalized = False
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-20 21:22:35,184] A new study created in memory with name: Study node type S, normalized: False


  0%|          | 0/25 [00:00<?, ?it/s]

[I 2023-12-20 21:24:03,524] Trial 0 finished with value: -100632.44213794377 and parameters: {'hidden_dim': 464, 'k': 3, 'learning_rate': 0.0008476504528272604, 'batcher': False}. Best is trial 0 with value: -100632.44213794377.
[I 2023-12-20 21:25:22,295] Trial 1 finished with value: 0.3081147964255764 and parameters: {'hidden_dim': 96, 'k': 1, 'learning_rate': 0.0019829951887567546, 'batcher': True}. Best is trial 1 with value: 0.3081147964255764.
[I 2023-12-20 21:26:48,102] Trial 2 finished with value: -10.064265177467956 and parameters: {'hidden_dim': 384, 'k': 3, 'learning_rate': 1.3344567351282657e-05, 'batcher': False}. Best is trial 1 with value: 0.3081147964255764.
[I 2023-12-20 21:28:15,662] Trial 3 finished with value: -10.639208210576568 and parameters: {'hidden_dim': 416, 'k': 3, 'learning_rate': 6.70057455786327e-05, 'batcher': False}. Best is trial 1 with value: 0.3081147964255764.
[I 2023-12-20 21:29:39,094] Trial 4 finished with value: -3.2358564699192165 and parameter

In [6]:
study.trials_dataframe().to_csv(f"csvs/single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}_s-{size_elem}.csv")

In [7]:
node_type = "S"
size_elem = 50
n_trials = 25
normalized = True
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-20 21:57:50,948] A new study created in memory with name: Study node type S, normalized: True


  0%|          | 0/25 [00:00<?, ?it/s]

[I 2023-12-20 21:59:19,764] Trial 0 finished with value: 0.8568392512233085 and parameters: {'hidden_dim': 240, 'k': 4, 'learning_rate': 0.009712530698295186, 'batcher': False}. Best is trial 0 with value: 0.8568392512233085.
[I 2023-12-20 22:00:52,899] Trial 1 finished with value: 0.8972972874464983 and parameters: {'hidden_dim': 336, 'k': 5, 'learning_rate': 0.005015008373117877, 'batcher': True}. Best is trial 1 with value: 0.8972972874464983.
[I 2023-12-20 22:02:31,029] Trial 2 finished with value: 0.8122714898739002 and parameters: {'hidden_dim': 272, 'k': 1, 'learning_rate': 6.17690426521759e-05, 'batcher': True}. Best is trial 1 with value: 0.8972972874464983.
[I 2023-12-20 22:04:00,008] Trial 3 finished with value: -787.5204744087289 and parameters: {'hidden_dim': 112, 'k': 5, 'learning_rate': 1.397562436600002e-05, 'batcher': True}. Best is trial 1 with value: 0.8972972874464983.
[I 2023-12-20 22:05:16,936] Trial 4 finished with value: -652.030284543117 and parameters: {'hidde

In [8]:
study.trials_dataframe().to_csv(f"csvs/single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}_s-{size_elem}.csv")

### Q

In [9]:
node_type = "Q"
size_elem = 50
n_trials = 25
normalized = False
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-20 22:34:38,671] A new study created in memory with name: Study node type Q, normalized: False


  0%|          | 0/25 [00:00<?, ?it/s]

[I 2023-12-20 22:36:18,913] Trial 0 finished with value: 0.6213004954341805 and parameters: {'hidden_dim': 240, 'k': 5, 'learning_rate': 0.0015743667839898021, 'batcher': False}. Best is trial 0 with value: 0.6213004954341805.
[I 2023-12-20 22:38:28,762] Trial 1 finished with value: -0.2577659993228114 and parameters: {'hidden_dim': 416, 'k': 3, 'learning_rate': 0.00021164311833902819, 'batcher': True}. Best is trial 0 with value: 0.6213004954341805.
[I 2023-12-20 22:40:33,267] Trial 2 finished with value: 0.161799750778922 and parameters: {'hidden_dim': 432, 'k': 3, 'learning_rate': 0.006789454842596385, 'batcher': True}. Best is trial 0 with value: 0.6213004954341805.
[I 2023-12-20 22:42:15,882] Trial 3 finished with value: 0.8154159947166375 and parameters: {'hidden_dim': 304, 'k': 3, 'learning_rate': 0.004600471266628685, 'batcher': True}. Best is trial 3 with value: 0.8154159947166375.
[I 2023-12-20 22:43:41,871] Trial 4 finished with value: -3.89741993536326 and parameters: {'hid

In [10]:
study.trials_dataframe().to_csv(f"csvs/single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}_s-{size_elem}.csv")

In [11]:
node_type = "Q"
size_elem = 50
n_trials = 25
normalized = True
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-20 23:17:22,402] A new study created in memory with name: Study node type Q, normalized: True


  0%|          | 0/25 [00:00<?, ?it/s]

[I 2023-12-20 23:18:49,642] Trial 0 finished with value: -355.0507489116784 and parameters: {'hidden_dim': 288, 'k': 3, 'learning_rate': 1.130842916384514e-05, 'batcher': False}. Best is trial 0 with value: -355.0507489116784.
[I 2023-12-20 23:20:14,674] Trial 1 finished with value: 0.9598863199771699 and parameters: {'hidden_dim': 336, 'k': 2, 'learning_rate': 0.0053118472656670575, 'batcher': False}. Best is trial 1 with value: 0.9598863199771699.
[I 2023-12-20 23:21:40,562] Trial 2 finished with value: 0.921111312956889 and parameters: {'hidden_dim': 320, 'k': 2, 'learning_rate': 0.006808124440791559, 'batcher': True}. Best is trial 1 with value: 0.9598863199771699.
[I 2023-12-20 23:23:02,372] Trial 3 finished with value: 0.9273591202348644 and parameters: {'hidden_dim': 304, 'k': 2, 'learning_rate': 0.0062507560904402435, 'batcher': True}. Best is trial 1 with value: 0.9598863199771699.
[I 2023-12-20 23:26:04,986] Trial 4 finished with value: -10.879799747157083 and parameters: {'h

In [12]:
study.trials_dataframe().to_csv(f"csvs/single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}_s-{size_elem}.csv")

### C

In [13]:
node_type = "C"
size_elem = 50
n_trials = 25
normalized = False
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-21 00:09:11,592] A new study created in memory with name: Study node type C, normalized: False


  0%|          | 0/25 [00:00<?, ?it/s]

[I 2023-12-21 00:11:33,696] Trial 0 finished with value: 0.5969359154620285 and parameters: {'hidden_dim': 400, 'k': 2, 'learning_rate': 0.000352340390973993, 'batcher': False}. Best is trial 0 with value: 0.5969359154620285.
[I 2023-12-21 00:13:41,024] Trial 1 finished with value: -0.31175435333656715 and parameters: {'hidden_dim': 256, 'k': 2, 'learning_rate': 1.9219053673733705e-05, 'batcher': False}. Best is trial 0 with value: 0.5969359154620285.
[I 2023-12-21 00:15:53,054] Trial 2 finished with value: 0.8913245228642177 and parameters: {'hidden_dim': 208, 'k': 1, 'learning_rate': 0.005883781965745241, 'batcher': True}. Best is trial 2 with value: 0.8913245228642177.
[I 2023-12-21 00:18:06,221] Trial 3 finished with value: 0.29938230766643315 and parameters: {'hidden_dim': 32, 'k': 3, 'learning_rate': 1.2355855810683269e-05, 'batcher': True}. Best is trial 2 with value: 0.8913245228642177.
[I 2023-12-21 00:21:51,409] Trial 4 finished with value: -15.114533331828 and parameters: {'

In [14]:
study.trials_dataframe().to_csv(f"csvs/single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}_s-{size_elem}.csv")

In [15]:
node_type = "C"
size_elem = 50
n_trials = 25
normalized = True
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-21 01:15:53,264] A new study created in memory with name: Study node type C, normalized: True


  0%|          | 0/25 [00:00<?, ?it/s]

[I 2023-12-21 01:17:57,426] Trial 0 finished with value: -964.3377997537311 and parameters: {'hidden_dim': 48, 'k': 5, 'learning_rate': 3.385880538204453e-05, 'batcher': True}. Best is trial 0 with value: -964.3377997537311.
[I 2023-12-21 01:20:37,211] Trial 1 finished with value: 0.8004943377436607 and parameters: {'hidden_dim': 320, 'k': 4, 'learning_rate': 0.00042784894143008465, 'batcher': False}. Best is trial 1 with value: 0.8004943377436607.
[I 2023-12-21 01:22:45,554] Trial 2 finished with value: -937.8409673280236 and parameters: {'hidden_dim': 112, 'k': 3, 'learning_rate': 1.103564668058445e-05, 'batcher': False}. Best is trial 1 with value: 0.8004943377436607.
[I 2023-12-21 01:24:52,734] Trial 3 finished with value: -15.90566126532853 and parameters: {'hidden_dim': 112, 'k': 5, 'learning_rate': 4.897711715622834e-05, 'batcher': False}. Best is trial 1 with value: 0.8004943377436607.
[I 2023-12-21 01:27:00,722] Trial 4 finished with value: 0.8863333153936803 and parameters: {

In [16]:
study.trials_dataframe().to_csv(f"csvs/single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}_s-{size_elem}.csv")

### P

In [17]:
node_type = "P"
size_elem = 50
n_trials = 25
normalized = False
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-21 02:15:40,980] A new study created in memory with name: Study node type P, normalized: False


  0%|          | 0/25 [00:00<?, ?it/s]

[I 2023-12-21 02:17:48,712] Trial 0 finished with value: 0.9931891163593656 and parameters: {'hidden_dim': 128, 'k': 5, 'learning_rate': 0.007132846848478785, 'batcher': True}. Best is trial 0 with value: 0.9931891163593656.
[I 2023-12-21 02:20:05,433] Trial 1 finished with value: 0.9964303475177589 and parameters: {'hidden_dim': 384, 'k': 2, 'learning_rate': 0.00018138184870426937, 'batcher': False}. Best is trial 1 with value: 0.9964303475177589.
[I 2023-12-21 02:22:16,304] Trial 2 finished with value: 0.996665576706586 and parameters: {'hidden_dim': 64, 'k': 1, 'learning_rate': 0.001960943758643364, 'batcher': True}. Best is trial 2 with value: 0.996665576706586.
[I 2023-12-21 02:24:23,292] Trial 3 finished with value: 0.9976954363388956 and parameters: {'hidden_dim': 496, 'k': 2, 'learning_rate': 4.428556788842352e-05, 'batcher': True}. Best is trial 3 with value: 0.9976954363388956.
[I 2023-12-21 02:26:35,721] Trial 4 finished with value: 0.9958754327612847 and parameters: {'hidde

In [18]:
study.trials_dataframe().to_csv(f"csvs/single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}_s-{size_elem}.csv")

In [19]:
node_type = "P"
size_elem = 50
n_trials = 25
normalized = True
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-21 03:16:23,791] A new study created in memory with name: Study node type P, normalized: True


  0%|          | 0/25 [00:00<?, ?it/s]

[I 2023-12-21 03:18:54,250] Trial 0 finished with value: 0.8643245847236305 and parameters: {'hidden_dim': 464, 'k': 5, 'learning_rate': 0.0010279678641954351, 'batcher': True}. Best is trial 0 with value: 0.8643245847236305.
[I 2023-12-21 03:21:00,446] Trial 1 finished with value: -950.3397200370609 and parameters: {'hidden_dim': 16, 'k': 3, 'learning_rate': 3.311241444316854e-05, 'batcher': True}. Best is trial 0 with value: 0.8643245847236305.
[I 2023-12-21 03:23:08,672] Trial 2 finished with value: 0.7121181788347923 and parameters: {'hidden_dim': 304, 'k': 1, 'learning_rate': 0.005595028993082658, 'batcher': False}. Best is trial 0 with value: 0.8643245847236305.
[I 2023-12-21 03:25:14,044] Trial 3 finished with value: 0.8389407299265279 and parameters: {'hidden_dim': 176, 'k': 4, 'learning_rate': 0.0019216300359400456, 'batcher': True}. Best is trial 0 with value: 0.8643245847236305.
[I 2023-12-21 03:27:21,803] Trial 4 finished with value: 0.7971064696273175 and parameters: {'hid

In [20]:
study.trials_dataframe().to_csv(f"csvs/single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}_s-{size_elem}.csv")